# Python Environment Setup

In [80]:
!pip3 install virtualenv

In [81]:
!python3 -m venv myenv

In [82]:
!source myenv/bin/activate

# This code will create a spreadsheet for all input, correct output and system output

In [210]:
import json
import pandas as pd

## creating the necessary columns as required

In [247]:
column_names = ["query_type","relation", "query","correct output", "system output","ssg output"]
my_data = pd.DataFrame(columns = column_names)

In [248]:
query_type = []
relation = []
query = []
correct = []
system = []
ssg_op = []


## This method will get query, answer, relation and query type from test file.

In [249]:
def get_data_from_original(path):
    with open(path) as json_file:
        for line in json_file:
            db = json.loads(line)
            for all_query in db["queries"]:
                query.append(all_query['query'])
                correct.append(all_query['answer'])
                relation.append(all_query['relation'])
                if "arg" in all_query['type'] or "min" in all_query['type'] or "max" in all_query['type']:
                    query_type.append("min/max")
                else:
                    query_type.append(all_query['type'])



In [250]:
get_data_from_original("Inference_original_onlytable/combined_data.jsonl")

## This function will get the gold facts and predicted facts from the predicted file

In [251]:
def get_data_from_prediction(path):
    with open(path) as json_file:
        db = json.load(json_file)

        for idx,data in enumerate(db):
            ssg_op.append(data["ssg_output"])
            system.append(data['gold_facts'])

In [252]:
get_data_from_prediction("Inference_original_onlytable/combined_data_0.7_ssg_sup.json")

In [253]:
my_data["query_type"] = query_type
my_data['query'] = query
my_data['relation'] = relation
my_data["correct output"] = correct
my_data["system output"] = system
my_data["ssg output"] = ssg_op

## We can analyze the correct output and system output to see whether their is an exact match or some partial match.

In [254]:
my_data.head()

,query_type,relation,query,correct output,system output,ssg output
0,min/max,P54,What is the smallest team?,"[Evagoras Paphos, Czech Republic, Albania, Ayi...","[[0], [1], [3], [4], [6]]","[[[0, Cypriot First Division team Evagoras Pap..."
1,bool,P54,Is UEFA Euro 2016 part of Omonia Aradippou?,[FALSE],"[[1], [2], [7]]","[[[0, Cypriot First Division team Evagoras Pap..."
2,count,P54,How many teams has UEFA Euro 2016 played for?,[6],"[[1], [3], [5], [6], [7], [9]]","[[[1, UEFA Euro 2016 team Czech Republic]], [[..."
3,set,P54,Who plays for Switzerland?,[UEFA Euro 2016],[[5]],"[[[5, UEFA Euro 2016 team Switzerland]], [[5, ..."
4,count,P54,How many teams has 2013 FIA World Endurance Ch...,[3],"[[1], [2], [4]]","[[[1, 2013 FIA World Endurance Championship te..."


## Save the dataframe as spreadsheet

In [255]:
my_data.to_csv('Inference_original_onlytable/Inference_original_onlytable_sheet_combined.tsv',sep='\t', index=False)